In [ ]:
import numpy as np
import pandas as pd
import math
import datetime as dt
import datapackage
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import multinomial
from typing import List
import seaborn as sns

## profit function ##

def profit_t(E,P,vc,fc,pi_tax,c_tax):
    pi = ((E*(P-vc-c_tax))-fc)*(1-pi_tax)
    return pi

## define State class

class State: 
    def __init__(self,mean,eta,lambda_switch,sigma):
        self.m = mean
        self.e = eta 
        self.l = lambda_switch
        self.s = sigma

## state characterisation
        
state_1 = State(50,0.44,0.26,0.23)
state_2 = State(30,1.05,0.28,0.44)

## lattice parameters ##

p_step = 1
s0_step = 5000

p_1 = 20
p_max = 100

s0_1 = 5000
s0_max = 20000

## lattice construction ##

P_grid = list(range(p_1,p_max,p_step))
s0_grid = list(range(s0_1,s0_max,s0_step))

## finite difference parameter definition ##

## parameter defn for P_i in i = 2,...,i_max-1 ##

def difference(P,p_step,sigma,eta,mean):
    a_i_central = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    b_i_central = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    a_i_forward = (sigma**2)*(P**2)/((p_step*2)*p_step)
    b_i_forward = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    a_i_backward = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    b_i_backward = (sigma**2)*(P**2)/((p_step*2)*p_step)
    if {a_i_central and b_i_central >= 0}:
        return [a_i_central,b_i_central]
    elif b_i_forward >= 0:
        return [a_i_forward,b_i_forward]
    else: 
        return [a_i_backward,b_i_backward]
    
## parameter defn for P_i in i = 1 ##
    
def difference_min(P,p_step,eta,mean):
    a_i = 0
    b_i = eta*(mean-P)/(p_step)
    return [a_i,b_i]

## parameter defn for P_i in i = i_max ##

def difference_max(P,p_step,eta,mean):
    a_i = eta*(mean - P)/(p_step)
    b_i = 0
    return [a_i,b_i]

def Profit(P,s_k):
    for i in range(n):
        if s_k > E:
            pi = profit_t(E,P,vc,fc,pi_tax,c_tax)
            return pi 
        elif s_k > 0:
            pi = profit_t(s_k,P,vc,fc,pi_tax,c_tax)
            return pi
        else:
            pi = 0
            return pi

In [ ]:
## characterise project ##

P = 60
n = 30
r = 0.05 
E = 5
vc = 10
fc = 0
pi_tax = 0.2
c_tax = 5

## create reserves matrix ##

s_r = []

## create s_k matrix ##

s_k = []

## create empty profit and valuation vectors ##


In [ ]:
## v_i_k defines iterative matrix of extraction option value for lattice nodes (P_i,S_k) 

v_i_k_new = np.zeros((len(P_grid),len(s0_grid)))
v_i_k_old = np.zeros((len(P_grid),len(s0_grid)))
Lv = np.zeros((len(P_grid),len(s0_grid)))

In [ ]:
## we solve option value recursively over a lease length of n ##

t = 1  
    
## first we loop over implied profit generation ##

## if implied profit (including fixed costs) exceeds fixed costs, then extraction occurs, else no extraction ##           

for i in range(len(P_grid)):
    for k in range(len(s0_grid)):
            
        if profit_t(E,P_grid[i],vc,fc,pi_tax,c_tax) > fc:                            
                
            if t*E <= s0_grid[k]:
                v_i_k_old[i,k] = v_i_k_old[i,k] + profit_t(E,P_grid[i],vc,fc,pi_tax,c_tax)
                
            elif {t*E - s0_grid[k] > 0}:
                v_i_k_old[i,k] = v_i_k_old[i,k] + profit_t(t*E - s0_grid[k],P_grid[i],vc,fc,pi_tax,c_tax)
                
            else:
                v_i_k_old[i,k] = v_i_k_old[i,k] + 0
        else:
            v_i_k_old[i,k] = v_i_k_old[i,k] - fc    

## then we loop over Lagrange differential for boundary nodes ##                

for k in range(len(s0_grid)):
    diff = difference_min(P_grid[0],p_step,state_1.e,state_1.m)
    Lv[0,k] = diff[1]*v_i_k_old[1,k] - (diff[0]+diff[1]+r)*v_i_k_old[0,k]

for k in range(len(s0_grid)):
    diff = difference_max(len(P_grid) - 1,p_step,state_1.e,state_1.m)
    Lv[len(P_grid) - 1,k] = diff[0]*v_i_k_old[len(P_grid) - 2,k]  - (diff[0]+diff[1]+r)*v_i_k_old[len(P_grid) - 1,k]

## then we loop over lagrange differential for nodes inside the boundary ##
    
for i in range(1,len(P_grid) - 1):
    for k in range(len(s0_grid)):
        diff = difference(P_grid[i],p_step,state_1.s,state_1.e,state_1.m)
        Lv[i,k] = diff[0]*v_i_k_old[i - 1,k] + diff[1]*v_i_k_old[i + 1,k] - (diff[0]+diff[1]+r)*v_i_k_old[i,k]


In [ ]:
print(Lv)
print(v_i_k_old)

In [ ]:
diff = difference(P_grid[10],p_step,state_1.s,state_1.e,state_1.m)
Lv = diff[0]*v_i_k_old[9,1] + diff[1]*v_i_k_old[11,1] - (diff[0]+diff[1]+r)*v_i_k_old[10,1]
print(diff)
print(Lv)

In [ ]:
print(state_1.m)
